In [60]:
# 하이퍼파라미터 설정
num_epochs = 10
learning_rate = 0.1

In [61]:
# 가중치 드랍아웃 레이어
class WeightDropout(nn.Module):
    def __init__(self, module, p=0.5):
        super(WeightDropout, self).__init__()
        self.module = module
        self.p = p
        self.weight = nn.Parameter(module.weight.data)

    def forward(self, *args):
        if self.training:
            # dropout을 적용한 가중치
            weight = F.dropout(self.weight, p=self.p, training=True)
            # self.module.weight에 새로운 가중치 할당
            self.module.weight = nn.Parameter(weight)
        return self.module(*args)

# ***Basic Code***

In [62]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
import torch.optim as optim
from torchvision.datasets import CIFAR10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [63]:
def dataset_MNIST():
  transform = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,))])
  train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
  test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
  return train_loader, test_loader


def dataset_CIFAR10():
  transform = transforms.Compose([
      transforms.Resize((32, 32)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])
  train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
  test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
  return train_loader, test_loader

In [64]:
# 모델 학습 함수
def train(num_epochs):
  model.to(device)
  for epoch in range(num_epochs):
      model.train()
      for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

      model.eval()
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
          accuracy = 100 * correct / total

      print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%')

# ***Experiments***

## [MNIST] basic setting (without Dropout)

In [65]:
class Net_MNIST(nn.Module):
    def __init__(self):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_MNIST()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_MNIST()

# 모델 학습
train(num_epochs)

Epoch [1/10], Test Accuracy: 98.15%
Epoch [2/10], Test Accuracy: 98.74%
Epoch [3/10], Test Accuracy: 98.92%
Epoch [4/10], Test Accuracy: 99.01%
Epoch [5/10], Test Accuracy: 99.09%
Epoch [6/10], Test Accuracy: 99.04%
Epoch [7/10], Test Accuracy: 99.04%
Epoch [8/10], Test Accuracy: 99.08%
Epoch [9/10], Test Accuracy: 99.09%
Epoch [10/10], Test Accuracy: 99.18%


## [MNIST] Dropout setting (with normal Dropout)

In [69]:
class Net_MNIST(nn.Module):
    def __init__(self):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500, 10) 
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_MNIST()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_MNIST()

# 모델 학습
train(num_epochs)

Epoch [1/10], Test Accuracy: 98.14%
Epoch [2/10], Test Accuracy: 98.71%
Epoch [3/10], Test Accuracy: 98.95%
Epoch [4/10], Test Accuracy: 98.98%
Epoch [5/10], Test Accuracy: 99.05%
Epoch [6/10], Test Accuracy: 99.13%
Epoch [7/10], Test Accuracy: 99.22%
Epoch [8/10], Test Accuracy: 99.20%
Epoch [9/10], Test Accuracy: 99.18%
Epoch [10/10], Test Accuracy: 99.17%


## [MNIST] our setting (with Weight Dropout)

In [66]:
class Net_MNIST(nn.Module):
    def __init__(self):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = WeightDropout(nn.Linear(500, 10), p=0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_MNIST()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_MNIST()

# 모델 학습
train(num_epochs)

Epoch [1/10], Test Accuracy: 96.98%
Epoch [2/10], Test Accuracy: 98.06%
Epoch [3/10], Test Accuracy: 98.47%
Epoch [4/10], Test Accuracy: 98.75%
Epoch [5/10], Test Accuracy: 98.89%
Epoch [6/10], Test Accuracy: 98.48%
Epoch [7/10], Test Accuracy: 98.87%
Epoch [8/10], Test Accuracy: 99.00%
Epoch [9/10], Test Accuracy: 99.02%
Epoch [10/10], Test Accuracy: 99.01%


## [CIFAR-10] basic setting (without Dropout)

In [70]:
class Net_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1)
        self.conv2 = nn.Conv2d(64, 128, 5, 1)
        self.fc1 = nn.Linear(5*5*128, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_CIFAR10()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_CIFAR10()

# 모델 학습
train(num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Test Accuracy: 35.96%
Epoch [2/10], Test Accuracy: 57.69%
Epoch [3/10], Test Accuracy: 64.51%
Epoch [4/10], Test Accuracy: 68.13%
Epoch [5/10], Test Accuracy: 71.52%
Epoch [6/10], Test Accuracy: 70.96%
Epoch [7/10], Test Accuracy: 71.03%
Epoch [8/10], Test Accuracy: 71.08%
Epoch [9/10], Test Accuracy: 72.19%
Epoch [10/10], Test Accuracy: 73.05%


## [CIFAR-10] Dropout setting (with normal Dropout)

In [71]:
class Net_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1)
        self.conv2 = nn.Conv2d(64, 128, 5, 1)
        self.fc1 = nn.Linear(5*5*128, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*128)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_CIFAR10()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_CIFAR10()

# 모델 학습
train(num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Test Accuracy: 42.55%
Epoch [2/10], Test Accuracy: 55.79%
Epoch [3/10], Test Accuracy: 56.83%
Epoch [4/10], Test Accuracy: 69.17%
Epoch [5/10], Test Accuracy: 70.10%
Epoch [6/10], Test Accuracy: 72.29%
Epoch [7/10], Test Accuracy: 72.53%
Epoch [8/10], Test Accuracy: 72.03%
Epoch [9/10], Test Accuracy: 74.84%
Epoch [10/10], Test Accuracy: 74.83%


## [CIFAR-10] our setting (with Weight Dropout)

In [75]:
class Net_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, 1)
        self.conv2 = nn.Conv2d(64, 128, 5, 1)
        self.fc1 = nn.Linear(5*5*128, 512)
        self.fc2 = WeightDropout(nn.Linear(512, 10), p=0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = Net_CIFAR10()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# 데이터셋 로드
train_loader, test_loader = dataset_CIFAR10()

# 모델 학습
train(num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Test Accuracy: 39.80%
Epoch [2/10], Test Accuracy: 53.78%
Epoch [3/10], Test Accuracy: 57.96%
Epoch [4/10], Test Accuracy: 64.68%
Epoch [5/10], Test Accuracy: 62.72%
Epoch [6/10], Test Accuracy: 64.27%
Epoch [7/10], Test Accuracy: 67.25%
Epoch [8/10], Test Accuracy: 71.33%
Epoch [9/10], Test Accuracy: 70.24%
Epoch [10/10], Test Accuracy: 73.03%
